In [ ]:
#This file is for visualizing web scraped data to understand food cateogry subtypes that can be derived from user reviews.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn import metrics
import re
import string
import time
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
##Constants
mystopwords=stopwords.words("english")+['singapore', 
'holiday', 'sister', 'brother', 'kid', 'generally', 'european', 
'typically', 'recommend', 'recommended', 'reasonable', 'like', 'liked',
'french', 'quite', 'back', 'definitely', 'place', 'food', 'sand', 'bay', 
'afternoon', 'home', 'lunch', 'dinner', 'roof', 'layout', 'restaurant',
'situation', 'set', 'email', 'rooftop', 'bar', 'really', 'good',
'work', 'music', 'marriot', 'skyline', 'hotel', 'impeccable', 'fullerton',
'classy', 'feel', 'presented', 'present', 'well', 'explaining', 'decoration',
'wife', 'husband', 'friend', 'perfectly', 'limited', 'called', 'call',
'delight', 'arrive', 'otherwise', 'unfortunately', 'enjoyable',
'seemed', 'managed', 'read', 'indeed', 'stay', 'great', 'plus',
'better', 'front', 'recently', 'floor', 'friday', 'monday', 'tuesday',
'wednesday', 'thursday', 'saturday', 'sunday', 'weekend', 'weekends',
'weekdays', 'weekday', 'often', 'wait', 'waited','seat', 'seated',
'week', 'ordering', 'something', 'next', 'know', 'known',
'helped', 'entrance', 'also', 'enough', 'reservation', 'going',
'go', 'need', 'coming', 'etc', 'sit', 'disappointing', 'certainly',
'able', 'gave', 'give', 'including', 'seems', 'table', 'must',
'try', 'time', 'would', 'will', 'even', 'many', 'especially', 'though',
'although', 'must try', 'year', 'outside', 'however', 'around', 'group',
'want', 'take', 'overall', 'took', 'surprised', 'recommendation',
'okay', 'let', 'either', 'bring', 'helpful', 'crowd', 'yes', 'no',
'except', 'soon', 'remember', 'business', 'queue', 'everyone',
'chose', 'station', 'fact', 'keep', 'anything', 'within','sgd',
'saw', 'expectation', 'due', 'stop', 'counter', 'corner', 'mine',
'check', 'hit', 'similar', 'despite', 'note', 'reasonably', 'impressed',
'impress', 'particularly', 'highlight', 'later', 'late', 'based',
'totally', 'heard', 'city', 'colleague', 'across', 'space', 'leave',
'world', 'avoid','various','tell', 'told', 'ended', 'town', 'change',
'personally', 'making', 'make', 'typical', 'help', 'outstanding',
'surprisingly', 'surprised', 'upon', 'into','nearby', 'please',
'pleased', 'pleasing', 'disappoint','followed','sharing', 'taken',
'took', 'take', 'among', 'amongst', 'sitting', 'sell', 'deal', 'party',
'one', 'two', 'menu','visit','get','view', 'marina', 'amazing',
'amaze', 'amazed', 'child', 'class', 'boy', 'complain', 'could',
'busy', 'choose', 'came', 'arrival', 'departure', 'could get',
'court', 'complimentary', 'boyfriend', 'girlfriend', 'charged',
'booking', 'book', 'brought', 'actually', 'absolutely', 'accompanied',
'addition', 'across', 'long', 'always', 'absolute', 'advance',
'acceptable', 'bright', 'ask', 'asking', 'base', 'almost', 'anyone',
'compare', 'compared', 'behind', 'case', 'country', 'away', 'arrangement',
'concept', 'apparently', 'anywhere', 'compliment', 'certain', 'available',
'charming', 'catch', 'attended', 'comfortable', 'ceiling', 'basically',
'booked', 'charge', 'card', 'combination', 'common', 'celebration', 
'closed', 'clarke', 'quay', 'chain', 'arrived', 'cheerful', 'clarke quay',
'average', 'carte', 'craving', 'branch', 'credit card', 'bought', 'bad',
'constantly', 'considering', 'believe', 'best', 'celebrate',
'chope', 'beautiful', 'buy', 'asked', 'anyway', 'ambiance', 'anniversary'
'ahead', 'additional', 'advice', 'abit',
'adult', 'alright', 'another', 'checking', 'birthday', 'bother',
'credit', 'compare', 'centre', '']

WNlemma = nltk.WordNetLemmatizer()
num_clusters = 5
vectorizer = TfidfVectorizer(max_df=0.3, max_features=6000,
                             min_df=5, stop_words=mystopwords,
                             use_idf=True, ngram_range=(1,3))
clusterHolder1 = []
clusterHolder2 = []

##Functions
def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens=[ t for t in tokens if False == t.isdigit()]
    tokens=[ t for t in tokens if False == containsNumeric(t)]
    tokens=[ t for t in tokens if True == t.isalpha()]
    tokens=[ t for t in tokens if t not in string.punctuation]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    text_after_process=" ".join(tokens)
    return(text_after_process)

def print_terms(cm, num):
    original_space_centroids = cm.cluster_centers_
    order_centroids = original_space_centroids.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(num):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

def containsNumeric(text):
    if text is not None:
        return any(str.isdigit(c) for c in text)
    else:
        return True

def saveClusterValues(model, clusterNum, array):
    original_space_centroids = model.cluster_centers_
    order_centroids = original_space_centroids.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for ind in order_centroids[clusterNum, :]:
        array.append(terms[ind])

In [ ]:
data_files = '../data/'
data = pd.read_csv('/content/drive/My Drive/PLP Sem 4/data.csv', encoding='unicode_escape')
data.groupby('restaurant')
text = data['Review']
tokens = text.apply(pre_process)
data.head()

,Unnamed: 0,Review,address,author,author_loc,date,origin,rating,restaurant,type,region
0,0,Visited on a day without Unagi *0*Location: 4/...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",ML-SG-001,"Singapore, Singapore",8/10/2020,TripAdvisor,2,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
1,1,"Since my visit to Sushi Kimura, I had this str...","7500A Beach Road The Plaza #B1-310, The Plaza,...",morsels-of-delice,"Singapore, Singapore",10/30/2019,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
2,2,We enjoyed our meal very much. The uni platter...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",baolover,"Singapore, Singapore",7/26/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
3,3,"Came here during Stage 2 CB period , Oosterbay...","7500A Beach Road The Plaza #B1-310, The Plaza,...",Tan T,"Singapore, Singapore",7/23/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL
4,4,A highly recommended good Japanese restaurant ...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",WSJJ,"Singapore, Singapore",7/3/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL


In [ ]:
X = vectorizer.fit_transform(tokens)
km1 = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=1000)

print("Fitting to k-means model...")
time_start = time.time()

km1.fit(X)
print('Fitting done! Time elapsed: {} seconds'.format(time.time()-time_start))

print("Coefficient for "+str(num_clusters)+" clusters: %0.3f"
     % metrics.silhouette_score(X, km1.labels_))

labels, counts = np.unique(km1.labels_[km1.labels_>=0], return_counts=True)
print (labels)
print (counts)
print_terms(km1, num_clusters)

Fitting to k-means model...
Fitting done! Time elapsed: 2849.6138598918915 seconds
Coefficient for 5 clusters: 0.004
[0 1 2 3 4]
[134749  18327  26296   7202  15959]
Cluster 0: dish price experience delicious meal chicken taste drink quality ordered
Cluster 1: nice ambience nice ambience staff drink nice service price atmosphere service nice wine
Cluster 2: staff friendly staff friendly attentive service staff friendly staff experience staff attentive delicious ambience
Cluster 3: crab chilli chilli crab pepper chili pepper crab chili crab seafood black pepper black
Cluster 4: excellent excellent service service excellent staff experience wine quality ambience highly dish


In [ ]:
df = pd.DataFrame(clusterHolder1)
df.to_csv('/content/drive/My Drive/PLP Sem 4/results.csv', index=False, header=['Food_SubType'])